# The purpose of this notebook is to explore the original dataset, and remove some unnecessary colomns and scale down the size of the file as much as possible to make it fit into a computer ram. 

In [11]:
import pandas as pd
import numpy as np

from composite_std import composite_SD

In [2]:
data = []
for i in range(10): # There are ten partitions in the transunion data.
    df = pd.read_csv("./data/transunion_{}.csv".format(i))
    df = df.loc[(df["IndustryCategory"] == "AT") & (df["IndustryVariableNum"] == 1)]
    data.append(df)

data = pd.concat(data)

In [4]:
data.head()

,Unnamed: 0,PostalCd,IndustryCategory,IndustryVariableNum,Count,Sum,Min,Max,Mean,Median,StdDev
17871,17871,P0R1C0,AT,1,15,166,2,24,11.07,10.0,7.01
17889,17889,P0R1C0,AT,1,15,128,1,22,8.53,7.0,5.49
17891,17891,V0N3A0,AT,1,18,216,1,35,12.00,9.5,9.75
17903,17903,J0P1Z0,AT,1,15,191,1,24,12.73,13.0,7.29
17912,17912,J0P1Z0,AT,1,16,193,1,29,12.06,11.0,8.90


In [8]:
duplicates_df = data[data.duplicated(["PostalCd"])]

In [10]:
duplicates_df.loc[duplicates_df["PostalCd"] == "A0G3K0"]

,Unnamed: 0,PostalCd,IndustryCategory,IndustryVariableNum,Count,Sum,Min,Max,Mean,Median,StdDev
32461318,4137,A0G3K0,AT,1,15,157,1,24,10.47,7.0,7.56
32461353,4172,A0G3K0,AT,1,15,182,1,30,12.13,12.0,7.81
32461720,4539,A0G3K0,AT,1,14,145,1,24,10.36,7.5,7.88
32461747,4566,A0G3K0,AT,1,12,93,1,19,7.75,6.5,5.17
32482237,25056,A0G3K0,AT,1,15,201,1,28,13.40,13.0,7.66
32482267,25086,A0G3K0,AT,1,15,167,1,37,11.13,9.0,10.20
32482308,25127,A0G3K0,AT,1,12,149,1,27,12.42,11.0,6.68
32482528,25347,A0G3K0,AT,1,15,139,2,18,9.27,8.0,4.89
32482561,25380,A0G3K0,AT,1,15,167,2,28,11.13,9.0,7.52
32499943,42762,A0G3K0,AT,1,14,102,1,20,7.29,6.5,6.04


# Postal code was further segregated into street level. So it needs to be aggregated into postal level.

In [12]:
def aggregate_group(x):
    d = {}
    d["Count"] = x["Count"].sum()
    d["Sum"] = x["Sum"].sum()
    d["Min"] = x["Min"].min()
    d["Max"] = x["Max"].max()
    d["Mean"] = d["Sum"] / d["Count"]
    d["Std"] = composite_SD(x["Mean"].values, x["StdDev"].values, x["Count"].values)
    return pd.Series(d)

agg_df = data.groupby("PostalCd").apply(aggregate_group)

/home/yang/.virtualenvs/sharpestmind/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in long_scalars
  import sys
/home/yang/Code/yang-sharpestminds/composite_std.py:24: RuntimeWarning: invalid value encountered in double_scalars
  GM /= N


In [22]:
agg_df = agg_df.reset_index()

In [27]:
agg_df.to_csv("./data/transunion_processed.csv")